In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,493.206238,-94.645447,-291.701904,601.198914,-32.923615,-662.838440,809.795105,-380.377197,825.971680,420.571716,...,194.174377,-267.659790,-500.838837,256.607788,408.753296,1069.934570,-142.697815,1,8077,critical
1,7.537169,-14.017939,-10.766094,16.649782,-1.662095,-11.386008,16.042494,-3.683760,25.835314,1.784303,...,-8.649843,-6.559853,-23.561625,23.530941,-1.597989,23.579197,-2.988943,1,5553,critical
2,0.231575,-9.463058,-18.066072,34.561588,-7.121534,-28.618713,19.281786,-5.490053,35.314980,12.210077,...,-12.267640,-5.113117,-20.395214,11.202926,-1.206951,24.665892,-19.308853,1,9341,critical
3,37.185074,-24.237020,-40.553955,42.658749,-3.337812,-17.803162,60.351562,-24.570055,50.746403,32.497662,...,6.340598,10.465193,-7.663659,28.312252,11.600004,40.185349,-49.591957,1,10419,critical
4,493.206238,-94.645447,-291.701904,601.198914,-32.923615,-662.838440,809.795105,-380.377197,825.971680,420.571716,...,194.174377,-267.659790,-500.838837,256.607788,408.753296,1069.934570,-142.697815,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8232
F1_macro: 0.7569
Accuracy: 0.8232
Precision_macro: 0.7816
Recall_macro: 0.7414

Bagging Metrics:
F1_micro: 0.8213
F1_macro: 0.7465
Accuracy: 0.8213
Precision_macro: 0.7850
Recall_macro: 0.7266

XGBoost Metrics:
F1_micro: 0.8209
F1_macro: 0.7605
Accuracy: 0.8209
Precision_macro: 0.7737
Recall_macro: 0.7510


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8151
F1_macro: 0.7531
Precision_macro: 0.7661
Recall_macro: 0.7433
Accuracy: 0.8151
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       814
           1       0.67      0.59      0.63       295

    accuracy                           0.82      1109
   macro avg       0.77      0.74      0.75      1109
weighted avg       0.81      0.82      0.81      1109


Bagging Metrics:
F1_micro: 0.8215
F1_macro: 0.7564
Precision_macro: 0.7780
Recall_macro: 0.7422
Accuracy: 0.8215
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       814
           1       0.70      0.57      0.63       295

    accuracy                           0.82      1109
   macro avg       0.78      0.74      0.76      1109
weighted avg       0.81      0.82      0.82      1109


XGBoost Metrics:
F1_micro: 0.8151
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/cubert/result_cubert_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/cubert/result_cubert_sum.pkl
